In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Per evitare l'overfitting è necessario fare del preprocessing alle immagini (detto image augmentation). Vengono applicate trasformazioni (shift, zoom, rotazioni) per modificare le immagini e renderle più varie.

target_size=dimensione dell'immagine dopo il resize
class_mode=il tipo di output aspettato, in questo caso binario essendo solo una classificazione 0 e 1.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode="binary")

Non viene applicata alcuna trasformazione al test set, solo la standardizzazione.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = train_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode="binary")

kernel_size = dimensione della matrice del filtro

filters = quanti filtri usare nella convoluzione. Ogni immagine sarà filtrata con n filtri

input_shate = nel caso di immagini, il primo layer deve specificare che tipo di input aspettarsi. In questo caso un immagine a colori 64x64px. Definisce anche l'input layer.

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(64,64,3))) # input layer

pool_size = dimensione della matrice, cioè l'area in cui viene misurato il massimo

strides = di quanto si sposta la matrice dopo aver calcolato il massimo.

In [ ]:

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Viene aggiunto un secondo layer con convoluzione e pooling

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Le matrici risultati vengono appiattite e diventano un singolo vettore

In [ ]:
cnn.add(tf.keras.layers.Flatten())

Essendo un network per computer vision più complesso di un semplice ANN, vengono usati più neuroni.

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # output layer 

In [ ]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_set, validation_data=test_set, epochs=25)

L'immagine viene caricata in formato PIL, ma il network richiede di avere un array.

L'immagine viene quindi trasformata in un array compatibile.
Inoltre, il CNN ha come input un batch di immagini, quindi è presente un ulteriore dimensione nel dataset che è appunto il batch di immagini.

Viene aggiunta una dimensione all'array con expand_dims. La dimensione viene aggiunta all'inizio perchè è corretto che la prima dimensione rappresenti il numero del batch e le altre dimensioni rappresentino l'immagine al suo interno.

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img("dataset/single_prediction/cat_or_dog_1.jpg", target_size=(64,64))
test_image = image.img_to_array(test_image)/255
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
print(train_set.class_indices)
# [batch][prediction] c'è solo un batch con all'interno l'unica immagine testata
if result[0][0] > 0.5:
  prediction = "dog"
else:
  prediction = "cat"

print(prediction)